# CSV file cleaning 

We import some packages for cleaning, plotting, and transformation.

In [1]:
%matplotlib notebook
from collections import defaultdict
import csv
import json
import pathlib
import re
from typing import NamedTuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Set up the path to the CSV data directory.

In [2]:
data_path = pathlib.Path('data/daily/csvs')
data_path

PosixPath('data/daily/csvs')

Construct a test path

In [3]:
test_fname = 'dly1616.csv'
test_fpath = data_path / test_fname
test_fpath

PosixPath('data/daily/csvs/dly1616.csv')

Look at the first 20 lines of the test file to get a sense of the structure of the file

In [4]:
!head -n20 'data/daily/csvs/dly1616.csv'

Station Name: COOLGREANEY (St.Martins)
Station Height: 67 M 
Latitude:52.760  ,Longitude: -6.239


date:  -  09 to 09 utc
rain:  -  Precipitation Amount (mm)
ind:   -  Indicator

date,ind,rain
01-apr-2003,0,0.200
02-apr-2003,0,0.000
03-apr-2003,0,0.000
04-apr-2003,0,0.000
05-apr-2003,0,0.000
06-apr-2003,0,0.000
07-apr-2003,0,0.000
08-apr-2003,0,0.000
09-apr-2003,0,0.000
10-apr-2003,0,0.000


Skip the irrelevant rows, and call `.head()` on the `DataFrame`

In [5]:
df = pd.read_csv(test_fpath, skiprows=9)
df.head()

,date,ind,rain
0,01-apr-2003,0,0.2
1,02-apr-2003,0,0.0
2,03-apr-2003,0,0.0
3,04-apr-2003,0,0.0
4,05-apr-2003,0,0.0


In [6]:
!head -n5 'data/header_rows.txt'

dly10023.csv:10
dly1024.csv:10
dly1033.csv:10
dly1042.csv:10
dly1043.csv:10


In [7]:
class FilenameHeaderLine(NamedTuple):
    filename: str
    line_num: int

with open('data/header_rows.txt', 'rt') as f:
    # create a list of tuples of (filename, header_line_num)
    fname_headerline = [tuple(line.strip().split(':')) for line in f]
    
    # turn this list of tuples into a list of NamedTuples
    header_starts = [
        FilenameHeaderLine(filename=fname, line_num=int(line_num))
        for fname, line_num in fname_headerline
    ]

header_starts[:2]

[FilenameHeaderLine(filename='dly10023.csv', line_num=10),
 FilenameHeaderLine(filename='dly1024.csv', line_num=10)]

In [8]:
class StationDetails(NamedTuple):
    id: int
    name: str
    county: str
    lat: float
    lon: float

with open('data/stations_clean.csv', 'rt') as f:
    csv_reader = csv.reader(f)

    # skip the header
    next(csv_reader)
    
    # create a list of NamedTuples with the details for each station
    details = [
        StationDetails(
            id=int(id),
            name=name,
            county=county,
            lat=float(lat),
            lon=float(lon),
        )
        for id, name, county, lat, lon in csv_reader
    ]

details[:2]

[StationDetails(id=2115, name='HACKETSTOWN_(Voc.Sch.)', county='Carlow', lat=52.857, lon=-6.552),
 StationDetails(id=375, name='Oak_Park', county='Carlow', lat=52.857, lon=-6.909)]

We now want to combine the station details with the information about the header lines

We can see that the two structures are of different length.
The `details` structure contains details for stations for which data wasn't available.
Create a blacklist, which ignores these stations.

In [9]:
print(len(details))
print(len(header_starts))

518
498


In [10]:
# these files can be identified by running the following command
# in the directory containing all of the downloaded zip files
# $ find . -type f -size -1024c
blacklist_string = """
./dly3422.zip
./dly9938.zip
./dly2931.zip
./dly538.zip
./dly9323.zip
./dly4413.zip
./dly3037.zip
./dly1807.zip
./dly4702.zip
./dly2604.zip
./dly199.zip
./dly5819.zip
./dly5602.zip
./dly9206.zip
./dly8912.zip
./dly2218.zip
./dly8123.zip
./dly5729.zip
./dly9106.zip
./dly907.zip
"""
# create a list of station numbers as ints
blacklist = [int(line[5:-4]) for line in blacklist_string.split()]
blacklist[:2]

[3422, 9938]

In [11]:
class StationFileDetails(NamedTuple):
    id: int
    name: str
    county: str
    lat: float
    lon: float
    fname: str
    header_line_num: int

# create a mapping from station number to filename and header line number
station_num_header_starts = {
    # extract station numbers from files in the form dly9106.csv
    int(re.match(r'dly(\d+)\.csv', fn_header_line.filename).group(1)): fn_header_line
    for fn_header_line in header_starts
}

station_and_file_details = []

for detail in details:
    # we're only interested in the `id` once we unpack `detail`
    station_id, *_ = detail
    
    # if `id` is in the blacklist then skip this record
    if station_id in blacklist:
        continue
    
    # otherwise, create a new super-structure with all
    # of the details
    d = StationFileDetails(
        *detail,
        *station_num_header_starts[station_id],
    )
    station_and_file_details.append(d)

station_and_file_details[0]

StationFileDetails(id=2115, name='HACKETSTOWN_(Voc.Sch.)', county='Carlow', lat=52.857, lon=-6.552, fname='dly2115.csv', header_line_num=10)

## Export to JSON

We now want to export the data structure (`station_and_file_details`) to JSON.

In [12]:
by_county = defaultdict(list)

for detail in station_and_file_details:
    d = {
        'id': detail.id,
        'station': detail.name,
        'filename': detail.fname,
        'lat': detail.lat,
        'lon': detail.lon,
        'header_line_num': detail.header_line_num,
    }

    by_county[detail.county].append(d)

by_county['Monaghan'][3]

{'id': 839,
 'station': 'EMYVALE',
 'filename': 'dly839.csv',
 'lat': 54.338,
 'lon': -6.956,
 'header_line_num': 10}

In [13]:
with open('data/stations.json', 'wt') as f:
    json.dump(by_county, f, indent=2)